In [1]:
from openai import OpenAI
import rag

openai_client = OpenAI()
index = rag.initialize_index()

Indexed 385 chunks from 95 documents


In [2]:
messages = [
    {"role": "user", "content": "hello"}
]

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)


In [6]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-D3MZoujubLVZx0D9ORxcXcZRRyn5P",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1769694140,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_1590f93f9d",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 8,
    "total_tokens": 17,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [9]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hello"}
]

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [14]:
system_prompt = """
You're an assistant that can make jokes. Always find out the name of
the person to make the jokes personalized. Once you know the name,
make the joke about them.
""".strip()

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "tell me a joke"}
]

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

print(response.choices[0].message.content)


Of course! But first, what's your name?


In [15]:
messages.append(response.choices[0].message)
messages.append({"role": "user", "content": "Alexey"})

In [17]:
response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

print(response.choices[0].message.content)

Great to meet you, Alexey! Here’s a joke just for you:

Why did Alexey bring a ladder to the bar?

Because he heard the drinks were on the house!


In [19]:
messages = [
    {"role": "user", "content": "Tell me a bedtime story (100 words)"}
]

stream = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    stream=True
)

for chunk in stream:
    delta = chunk.choices[0].delta
    if delta.content:
        print(delta.content, end='')


Once in a serene forest, a little fox named Finn discovered a hidden glade glowing under the moonlight. Curious, he tiptoed closer and found a circle of dancing fireflies. They beckoned him to join, illuminating the night with their warm glow. Finn danced and twirled, forgetting his worries, embraced by joy. As the night deepened, the fireflies taught him the secret of dreams: close your eyes, believe in magic, and anything is possible. Finn thanked his new friends and promised to return. With his heart full of wonder, he drifted to sleep, dreaming of starry adventures in the enchanted forest.

In [20]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [22]:
messages = [
    {"role": "system", "content": "Extract the event information."},
    {
        "role": "user",
        "content": "Alice and Bob are going to a science fair on Friday.",
    },
]

response = openai_client.chat.completions.parse(
    model='gpt-4o-mini',
    messages=messages,
    response_format=CalendarEvent
)

In [24]:
response.choices[0].message.parsed

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [31]:
class ChatCompletionsRAG(rag.RAG):

    def llm(self, user_prompt, model="gpt-4o-mini"):
        messages = [
            {"role": "system", "content": self.rag_instructions},
            {"role": "user", "content": user_prompt}
        ]

        response = self.llm_client.chat.completions.parse(
            model=model,
            messages=messages,
            response_format=self.output_type
        )

        return response.choices[0].message.parsed


In [32]:
responses_rag = ChatCompletionsRAG(index, openai_client)
response = responses_rag.rag("llm as a judge")
print(response.answer[:50] + "...")

In the context of using a Large Language Model (LL...


In [34]:
import os
from openai import OpenAI

groq_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY")
)